In [9]:
import yt_dlp
import concurrent.futures

In [10]:
def download_youtube_video(url, save_path="./", cookies_path="cookies.txt"):
    try:
        ydl_opts = {
            'outtmpl': f"{save_path}/%(title)s.%(ext)s",
            'format': 'bestvideo+bestaudio/best',
            # 'cookies': cookies_path  # Provide the cookies file
            'cookiesfrombrowser': ('chrome',)
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        print("Download completed!")
    except Exception as e:
        print(f"An error occurred: {e}")

In [11]:
def get_latest_videos_from_channel(channel_url, playlist_items = '1-2'):
    ydl_opts = {
        'quiet': False,  # 显示调试信息，帮助我们理解问题
        'extract_flat': True,  # 只提取视频信息，不下载视频
        'playlist_items': playlist_items, 
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            # 提取频道信息
            info_dict = ydl.extract_info(channel_url, download=False)

            # 获取视频条目列表
            videos = info_dict.get('entries', [])

            if not videos:
                print("No videos found.")
                return []

            # 提取所有视频链接
            video_links = [
                video['url'] for video in videos
                if video.get('duration', 0) <= 600  # 600秒等于10分钟
            ]
            
            return video_links

        except Exception as e:
            print(f"Error occurred: {e}")
            return []

In [12]:
def download_latest_videos_concurrently(channel_url, playlist_items='1-2'):
    """获取并并行下载频道的最新视频"""
    video_urls = get_latest_videos_from_channel(channel_url, playlist_items)
    
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(download_youtube_video, video_url) for video_url in video_urls]
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()
            except Exception as e:
                print(f"An error occurred during download: {e}")

In [13]:
import subprocess

def process_video(input_file, output_file):
    """
    使用 FFmpeg 对视频进行以下处理：
    1. 移除元数据。
    2. 调整分辨率到 1280x720。
    3. 调整帧率到 30fps。
    4. 添加轻微的亮度和对比度调整。
    5. 添加随机噪声。
    """
    # 创建 FFmpeg 命令
    command = [
        "ffmpeg", "-i", input_file,  # 输入文件
        # 移除元数据
        "-map_metadata", "-1",
        # 视频滤镜：调整分辨率、帧率、亮度、对比度和添加噪声
        # "-vf", "scale=1280:720,eq=brightness=0.05:contrast=1.1,noise=alls=10:allf=t+u",
        "-vf", "scale=1280:720,eq=brightness=0.05:contrast=1.1",
        # 设置帧率
        "-r", "30",
        # 视频编码器和参数
        "-c:v", "libx264", "-crf", "18", "-b:v", "3M",
        "-preset", "slow",
        # 音频编码器
        "-c:a", "aac",
        # 输出文件
        output_file
    ]

    # 执行 FFmpeg 命令
    try:
        subprocess.run(command, check=True)
        print(f"视频处理完成，保存为: {output_file}")
    except subprocess.CalledProcessError as e:
        print(f"处理视频时出错: {e}")



In [16]:

channel_url = 'https://www.youtube.com/@NBA/videos'
download_latest_videos_concurrently(channel_url, playlist_items='1-2')

[youtube:tab] Extracting URL: https://www.youtube.com/@NBA/videos
[youtube:tab] @NBA/videos: Downloading webpage
[download] Downloading playlist: NBA - Videos
[youtube:tab] Playlist NBA - Videos: Downloading 2 items
[download] Downloading item 1 of 2
[download] Downloading item 2 of 2
[download] Finished downloading playlist: NBA - Videos
Extracting cookies from chrome
Extracted 2877 cookies from chrome
[youtube] Extracting URL: https://www.youtube.com/watch?v=7LYnNhyUfgg
[youtube] 7LYnNhyUfgg: Downloading webpage
[youtube] 7LYnNhyUfgg: Downloading ios player API JSON
[youtube] 7LYnNhyUfgg: Downloading mweb player API JSON


ERROR: [youtube] 7LYnNhyUfgg: Sign in to confirm you’re not a bot. This helps protect our community. Learn more


An error occurred: ERROR: [youtube] 7LYnNhyUfgg: Sign in to confirm you’re not a bot. This helps protect our community. Learn more


In [7]:
import concurrent.futures

def process_webm_files_concurrently(directory):
    # 获取指定目录下所有的 .webm 文件
    webm_files = [f for f in os.listdir(directory) if f.endswith('.webm')]

    # 使用并行计算处理每个 .webm 文件
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {
            executor.submit(process_video, input_file, os.path.splitext(input_file)[0] + ".mp4"): input_file
            for input_file in webm_files
        }
        for future in concurrent.futures.as_completed(futures):
            input_file = futures[future]
            try:
                future.result()
                print(f"{input_file} 处理完成")
            except Exception as e:
                print(f"处理 {input_file} 时出错: {e}")

In [8]:
import os

current_directory = os.getcwd()
process_webm_files_concurrently(current_directory)

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developersffmpeg version 7.1ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers Copyright (c) 2000-2024 the FFmpeg developers


  built with Apple clang version 16.0.0 (clang-1600.0.26.4)
  built with Apple clang version 16.0.0 (clang-1600.0.26.4)
  built with Apple clang version 16.0.0 (clang-1600.0.26.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1_3 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --en

视频处理完成，保存为: NBA’s Top 10 Plays of the Night ｜ November 19, 2024.mp4
NBA’s Top 10 Plays of the Night ｜ November 19, 2024.webm 处理完成


[out#0/mp4 @ 0x124704750] video:126371KiB audio:2607KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.136989%
frame= 4961 fps= 15 q=-1.0 Lsize=  129155KiB time=00:02:45.30 bitrate=6400.7kbits/s dup=5 drop=0 speed=0.499x    
[libx264 @ 0x124705640] frame I:34    Avg QP:16.36  size:175546
[libx264 @ 0x124705640] frame P:1595  Avg QP:19.42  size: 51111
[libx264 @ 0x124705640] frame B:3332  Avg QP:24.54  size: 12579
[libx264 @ 0x124705640] consecutive B-frames:  7.7%  6.1%  6.2% 80.0%
[libx264 @ 0x124705640] mb I  I16..4:  2.8% 60.3% 36.9%
[libx264 @ 0x124705640] mb P  I16..4:  2.6% 15.2%  2.9%  P16..4: 33.4% 23.8% 13.8%  0.0%  0.0%    skip: 8.3%
[libx264 @ 0x124705640] mb B  I16..4:  0.3%  1.0%  0.3%  B16..8: 40.0% 11.5%  3.6%  direct: 4.1%  skip:39.1%  L0:43.0% L1:42.4% BI:14.6%
[libx264 @ 0x124705640] 8x8 transform intra:70.7% inter:52.1%
[libx264 @ 0x124705640] direct mvs  spatial:99.8% temporal:0.2%
[libx264 @ 0x124705640] coded y,uvDC,uvAC intra: 62.9% 74.1%

视频处理完成，保存为: Dalton Knecht Matches NBA Record In Career-High Performance 🏆🔥 ｜ November 19, 2024.mp4
Dalton Knecht Matches NBA Record In Career-High Performance 🏆🔥 ｜ November 19, 2024.webm 处理完成


视频处理完成，保存为: Dalton Knecht Scores 5 Straight Threes In The 3rd UNCUT 🏆 ｜ November 19, 2024.mp4
Dalton Knecht Scores 5 Straight Threes In The 3rd UNCUT 🏆 ｜ November 19, 2024.webm 处理完成


[out#0/mp4 @ 0x150e15130] video:210126KiB audio:5637KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.176148%
frame=10697 fps= 24 q=-1.0 Lsize=  216143KiB time=00:05:56.50 bitrate=4966.7kbits/s dup=11 drop=0 speed=0.804x    
[libx264 @ 0x150e159e0] frame I:52    Avg QP:15.89  size:153994
[libx264 @ 0x150e159e0] frame P:3423  Avg QP:18.91  size: 41702
[libx264 @ 0x150e159e0] frame B:7222  Avg QP:24.18  size:  8919
[libx264 @ 0x150e159e0] consecutive B-frames:  6.7%  8.1%  5.5% 79.8%
[libx264 @ 0x150e159e0] mb I  I16..4:  4.5% 50.6% 44.9%
[libx264 @ 0x150e159e0] mb P  I16..4:  3.0% 12.9%  2.8%  P16..4: 33.0% 22.6% 14.6%  0.0%  0.0%    skip:11.0%
[libx264 @ 0x150e159e0] mb B  I16..4:  0.4%  0.9%  0.2%  B16..8: 34.9%  8.8%  2.7%  direct: 3.4%  skip:48.7%  L0:39.1% L1:44.6% BI:16.3%
[libx264 @ 0x150e159e0] 8x8 transform intra:66.3% inter:48.0%
[libx264 @ 0x150e159e0] direct mvs  spatial:99.9% temporal:0.1%
[libx264 @ 0x150e159e0] coded y,uvDC,uvAC intra: 55.0% 68.6